## 2nd. Actors and Blueprints
Actors not ony include vehicles and walkers, but also sensors, traffic signs, traffic lights, and the spectator.

# Blueprints

In [1]:
import sys
print(sys.version)
import carla

3.7.4 (default, Dec  8 2020, 11:37:07) 
[GCC 7.5.0]


In [2]:
client = carla.Client('localhost',2000)
client.set_timeout(10.0) # seconds
world = client.get_world()

**Managing the blueprint library**
 : The carla.BlueprintLibray class contains a list of carla.ActorBlueprint elements. It is the world object who can provide access to it.

In [21]:
import random
# Find a specific blueprint.
# Blueprints has an ID to identify them and
#  the actors spawned with it
# The library can be read to find a certain ID
blueprint_library = world.get_blueprint_library()
collision_sensor_bp = blueprint_library.find('sensor.other.collision')

# Choose a vehicle blueprint at random.
vehicle_bp = random.choice(blueprint_library.filter('vehicle.*.*'))

Spawned vehicle has 4 wheels


In [23]:
# Check vehicle attribute
number_of_wheels = int(vehicle_bp.get_attribute('number_of_wheels'))
print('Spawned vehicle has {} wheels'.format(number_of_wheels))
is_bike = [number_of_wheels == 2]
if(is_bike):
    vehicle_bp.set_attribute('color', '255,0,0')
else:
    vehicle_bp.set_attribute('color', '255,255,255')

Spawned vehicle has 4 wheels


In [24]:
for attr in vehicle_bp:
    if attr.is_modifiable:
        vehicle_bp.set_attribute(attr.id, random.choice(attr.recommended_values))

## Actor life cycle
**Spawing**
 : The world object is responsible of spawning actors and keeping track of these. CARLA uses the Unreal Engine coordinate system. carla.Rotation constructor is defined as (pitch, yaw, roll), that differs from Unreal Engine Editor (roll, pitch, yaw)

In [7]:
transform = carla.Transform(carla.Location(x=230, y=195, z=40), carla.Rotation(yaw=180))
actor = world.spawn_actor(vehicle_bp, transform)

In [31]:
# map.get_spawn_points()
# : for vehicles, returns a list of recommended spawning points
spawn_points = world.get_map().get_spawn_points()

AttributeError: 'list' object has no attribute 'location'

In [33]:
# world.get_random_location()
# : for walkers, returns a random point on a sidewalk.
# : this same method is used to set a goal location for walkers
spawn_point = carla.Transform()
spawn_point.location = world.get_random_location_from_navigation()
print(spawn_point.location)

Location(x=144.018509, y=205.113708, z=0.166550)


In [36]:
actor_list = world.get_actors()
# Find an actor by id. --> where is id defined at ?
actor = actor_list.find(id)
# Print the location of all the speed limit signs in the world.
for speed_sign in actor_list.filter('traffic.speed_limit.*'):
    print(speed_sign.get_location())

ArgumentError: Python argument types in
    ActorList.find(ActorList, builtin_function_or_method)
did not match C++ signature:
    find(carla::client::ActorList {lvalue}, unsigned int id)

**Handling**
 : carla.Actor mostly constists of get() and set() methods to manage the actors around the map

In [43]:
# show actor's acc. and velocity
# and immediately pick it up
print(actor.get_acceleration())
print(actor.get_velocity())

location = actor.get_location()
location.z += 10.0
actor.set_location(location)

Vector3D(x=-0.006779, y=-0.000152, z=0.036554)
Vector3D(x=0.713717, y=0.015046, z=0.000321)


In [44]:
# The actor's physics can be disabled to freeze it in place
actor.set_simulate_physics(False)

## Types of actors
**Sensors**
Sensors are actors that produce a stream of data. They have their own section
(**Refer to 4th. Sensors and data**)

In [51]:
camera_bp = blueprint_library.find('sensor.camera.rgb')
relative_transform = carla.Transform(carla.Location(x=10, y=0, z=5),
                                     carla.Rotation(yaw=180))
camera = world.spawn_actor(camera_bp,
                           relative_transform,
                           attach_to=actor)
camera.listen(lambda image:
              image.save_to_disk('output/%06d.png' % image.frame))
# and must destroy camera blueprint
# that bp continues to take some pictures.

In [52]:
camera.destroy()

False

In [ ]:
# attaches a camera rigidly to a vehicle, so their relative position remains fixed
camera_bp = blueprint_library.find('sensor.camera.rgb')
relative_transform = carla.Transform(carla.Location(x=10, y=0, z=5),
                                     carla.Rotation(yaw=180))
camera = world.spawn_actor(camera_bp,
                           relative_transform,
                           attach_to=my_vehicle,
                           carla.AttachmentType.Rigid)

**Spectator**
 : Placed by Unreal Engine to provide an in-game point of view. It can be used to move the view of the simulator window.

In [55]:
spectator = world.get_spectator()
transform = actor.get_transform()
spectator.set_transform(carla.Transform(transform.location +
                                        carla.Location(z=5),
                                        carla.Rotation(pitch=-90)))

**Traffic signs and traffic lights**
 : Only stops, yields and traffic lights are considered actors in CARLA so far. The rest of the OpenDRIVE signs are accessible from the API as carla.Landmark -> go to (**3rd. Maps and navigation**)

**Vehicles**
 : carla.Vehicle are a special type of actor. This is achieved applying 4 types of different controls.

In [66]:
# Release simulate physics
# actor.set_simulate_physics(False)
actor.set_simulate_physics(True)
actor.apply_control(carla.VehicleControl(throttle=1.0, steer=-1.0))

In [60]:
# carla.VehiclePhysicsControl defines
#  physical attributes of the vehicle.
# Besides many different attribute, this controller contains
#  two more controllers. carla.GearPhysicsControl for the gears.
# The other is a list of carla.WheelPhysicsControl,
#  that provide specific control over the different wheels. 
actor.apply_physics_control(
    carla.VehiclePhysicsControl(
        max_rpm = 5000.0,
        center_of_mass = carla.Vector3D(0.0, 0.0, 0.0),
        torque_curve=[[0,400],[5000,400]]))

In [64]:
# The autopilot mode will subscribe them to the Traffic manager,
#  and simulate real urban conditions. This module is hard-coded,
#  not based on machine learning. 
actor.set_simulate_physics(True)
actor.set_autopilot(True)

**Bounding Box**
 : In order to apply physics and detect collisions, vehicles have a carla.BoundingBox encapsulating them. 

In [69]:
box = actor.bounding_box
print(box.location)         # Location relative to the vehicle.
print(box.extent)           # XYZ half-box extents in meters.

Location(x=-0.000000, y=-0.003242, z=1.233721)
Vector3D(x=2.601919, y=1.307286, z=1.233722)


**Walkers**
 : I didnt check below lines

In [73]:
walker_controller_bp = world.get_blueprint_library().find('controller.ai.walker')
world.spawn_actor(walker_controller_bp, carla.Transform())

In [74]:
ai_controller.start()
ai_controller.go_to_location(world.get_random_location_from_navigation())
ai_controller.set_max_speed(1 + random.random())  # Between 1 and 2 m/s (default is 1.4 m/s).

NameError: name 'ai_controller' is not defined

In [75]:
ai_controller.stop()

NameError: name 'ai_controller' is not defined